# Graph-based Design Smells Analysis

In the GitHub-Repository https://github.com/feststelltaste/designsmells, there are many hidden design smells in the Java code.

The following code tries to find some of them.

You can directly login into Neot4j's

_For the database schema, please take a look at http://jqassistant.github.io/jqassistant/doc/1.8.0/#_java_plugin _


## Establish connection
Credentials and connection to the pre-loaded Neo4j database (read-only mode).

You can also use the credentials to directly login into the Neo4j database's frontend.
More details for the login see here https://github.com/feststelltaste/designsmells#online-version 

In [1]:
from py2neo import Graph

url = "bolt://hobby-dhecfkdfpfnpgbkefmfnhael.dbs.graphenedb.com:24787"
user = "user"
password= "b.cKOL3KxJO2aW.Ivdq38GXw8SnFVJl"

graph = Graph(
    url,
    auth=(user, password),
    secure=True)

graph.run("MATCH ()-[r]->() RETURN count (r) as count").to_data_frame()

,count
0,1305


## Find some design smells

### God Class
A class or interface with very many methods.

In [2]:
query = """
MATCH (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Class, COUNT(m) as NumberOfMethods
ORDER BY NumberOfMethods DESC
LIMIT 1
"""
graph.run(query).to_data_frame()

,Class,NumberOfMethods
0,de.feststelltaste.abstraction.god.God,20


### Complex Hierarchy 
A class whose inheritance hierarchy goes very deep.


In [3]:
query = """
MATCH
  h=(c:Class)-[:EXTENDS*]->(t:Type)
RETURN
  c.name as Class, length(h) as Depth
ORDER BY
  Depth desc
LIMIT 5
"""
graph.run(query).to_data_frame()

,Class,Depth
0,NewDynamicPortalableMvcView2,8
1,NewDynamicPortalableMvcView,7
2,NewDynamicPortalableMvcView2,7
3,DynamicPortalableMvcView,6
4,NewDynamicPortalableMvcView,6


### Deficient Encapsulation
A class declares public fields.


In [4]:
query = """
MATCH
  (class:Class)-[declares:DECLARES]->(field:Field)
WHERE
  field.visibility = 'public'
RETURN
  class.fqn AS Class, COUNT(declares) AS `public fields`
  """
graph.run(query).to_data_frame()

,Class,public fields
0,de.feststelltaste.encapsulation.glass.GlassClass,5
